## knock 035 検索結果を条件指定に使った副問い合わせを行う

In [15]:
import polars as pl
pl.Config.set_tbl_cols(-1)# 列が省略されないようにする

polars.config.Config

### データを読み込む

In [16]:
df_receipt = pl.read_csv("../docker/work/data/receipt.csv")
df_receipt.head()

sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
i64,i64,str,i64,i64,str,str,i64,i64
20181103,1541203200,"""S14006""",112,1,"""CS006214000001…","""P070305012""",1,158
20181118,1542499200,"""S13008""",1132,2,"""CS008415000097…","""P070701017""",1,81
20170712,1499817600,"""S14028""",1102,1,"""CS028414000014…","""P060101005""",1,170
20190205,1549324800,"""S14042""",1132,1,"""ZZ000000000000…","""P050301001""",1,25
20180821,1534809600,"""S14025""",1102,2,"""CS025415000050…","""P060102007""",1,90


In [17]:
df_receipt[0, "customer_id"]

'CS006214000001'

### ノック

In [18]:
(
    df_receipt
    .filter( ~pl.col("customer_id").str.starts_with("Z") )
    .group_by(by = "customer_id")
    .agg(pl.sum("amount"))
    .filter( pl.col("amount") > pl.mean("amount") )
    .sort(by = "customer_id")
    #.head(n = 10)
    # テキストのIDを抽出する
    .filter((pl.col("customer_id") == "CS029212000033") |
            (pl.col("customer_id") == "CS007515000119") |
            (pl.col("customer_id") == "CS034515000123") |
            (pl.col("customer_id") == "CS026414000014") |
            (pl.col("customer_id") == "CS07615000070") |
            
            (pl.col("customer_id") == "CS016414000063") |
            (pl.col("customer_id") == "CS012514000018") |
            (pl.col("customer_id") == "CS029515000142") |
            (pl.col("customer_id") == "CS015215000021") |
            (pl.col("customer_id") == "CS039814000011"))
)

customer_id,amount
str,i64
"""CS007515000119…",7157
"""CS012514000018…",2562
"""CS015215000021…",3090
"""CS016414000063…",6207
"""CS026414000014…",6671
"""CS029212000033…",3604
"""CS029515000142…",3420
"""CS034515000123…",3699
"""CS039814000011…",8031
